# Day 2: Data tools (Part 1)


In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


## `pandas`

pandas is the primary tool for working with tabular data in Python.

It provides two main data structures: **Series** (1-dimensional) and **DataFrame** (2-dimensional).

### DataFrame basics

You can think of a DataFrame as a "super-powered" spreadsheet - It has a notion of labeled rows and columns with values in each slot.

Let's start by putting some US unemployment data by region into a DataFrame.

In [ ]:
# Create sample data
years = list(range(1995, 2017, 2))
data = {
    "NorthEast": [5.9, 5.6, 4.4, 3.8, 5.8, 4.9, 4.3, 7.1, 8.3, 7.9, 5.7],
    "MidWest": [4.5, 4.3, 3.6, 4.0, 5.7, 5.7, 4.9, 8.1, 8.7, 7.4, 5.1],
    "South": [5.3, 5.2, 4.2, 4.0, 5.7, 5.2, 4.3, 7.6, 9.1, 7.4, 5.5],
    "West": [6.6, 6.0, 5.2, 4.6, 6.5, 5.5, 4.5, 8.6, 10.7, 8.5, 6.1],
    "National": [5.6, 5.3, 4.3, 4.2, 5.8, 5.3, 4.6, 7.8, 9.1, 8., 5.7]
}

unemp = pd.DataFrame(data, index=years)
unemp.head()

**Key visual operations:**

- `.head()` and `.tail()`: View first/last rows
- `.describe()`: Summary statistics
- `.dtypes()`: Type information

There are lots of other useful operations that you will discover on your own as you work on the practice problems. One of the easiest ways to do this will be by interfacing with your favorite LLM.

In [ ]:
print("\nSummary statistics:")
print(unemp.describe())

**Selecting values from a DataFrame**

Two main ways of selecting

- `.loc[row, col]`: Select by label
- `.iloc[row, col]`: Select by position

In [ ]:
unemp.loc[2009, "MidWest"]

In [ ]:
unemp.loc[2009, :]

In [ ]:
unemp.loc[1995, ["NorthEast", "National"]]

In [ ]:
unemp.loc[:, ["NorthEast", "National"]]

In [ ]:
unemp.loc[[1995, 2005, 2015], ["NorthEast", "National"]]

**Computation on columns**

You often want to operate on a certain columns of data at a time or combine them

In [ ]:
unemp["West"] / 100

In [ ]:
unemp["West"].max()

In [ ]:
unemp["West"] - unemp["MidWest"]

In [ ]:
unemp["West"].corr(unemp["MidWest"])

In [ ]:
unemp.corr()

In [ ]:
# `eval` is extra efficient because it doesn't create as many copies
unemp.eval("West / 100")

**Datatypes**

A handful of different data types can be stored in a DataFrame

- Booleans (`bool`)  
- Floating point numbers (`float64`)  
- Integers (`int64`)  
- Dates (`datetime`) — we will learn this soon  
- Categorical data (`categorical`)  
- Everything else, including strings (`object`)

We will typically refer to the type of data stored in a column as its `dtype`.

In [ ]:
str_unemp = unemp.copy()
str_unemp["South"] = str_unemp["South"].astype(str)
str_unemp.dtypes

Everything looks okay if we examine `str_unemp`

In [ ]:
str_unemp.head()

but...

In [ ]:
str_unemp.sum()

**Creating new columns**

We can create new data by assigning values to a column similar to how we assign values to a variable.

In pandas, we create a new column of a DataFrame by writing:

```python
df["New Column Name"] = new_values
```


In [ ]:
unemp["UnweightedMean"] = (
    unemp["NorthEast"] +
    unemp["MidWest"] +
    unemp["South"] +
    unemp["West"]
) / 4

In [ ]:
unemp.head()

You can also change a particular value (or set of values) and rename columns/indexes

In [ ]:
unemp.loc[1995, "NorthEast"] = 6.0

In [ ]:
new_unemp = unemp.rename(
    columns={
        "NorthEast": "northeast",
        "MidWest": "midwest",
        "South": "south",
        "West": "west",
        "National": "national",
        "UnweightedMean": "unweightedmean"
    }
)

new_unemp.head()

### Datetime objects

As economists, we often care about working with date objects since we're interested in how things change over time.

Luckily for us, they are first-class citizens in the `pandas` ecosystem.

In [ ]:
# Load up slightly larger data
url = "https://datascience.quantecon.org/assets/data/state_unemployment.csv"
unemp_raw = pd.read_csv(url, parse_dates=["Date"])

# Don't worry about the details here quite yet
unemp_all = (
    unemp_raw
    .reset_index()
    .pivot_table(index="Date", columns="state", values="UnemploymentRate")
)

states = [
    "Arizona", "California", "Florida", "Illinois",
    "Michigan", "New York", "Texas"
]
unemp = unemp_all[states]
unemp.head()

In [ ]:
unemp.plot()

**Selecting subset of dates**

There are "special" ways to do selection on dates - You can always use the standard library's `datetime` type as seen below, but you can also treat dates as strings and they will get automatically converted

In [ ]:
unemp.loc[dt.datetime(2000, 1, 1), :]

In [ ]:
# Data corresponding to a single date
unemp.loc["2000-01-01", :]

In [ ]:
# Data for all days between New Years Day and June first in the year 2000
unemp.loc["01/01/2000":"06/01/2000", :]

### Aggregations

Let’s talk about *aggregations*.

Loosely speaking, an aggregation is an operation that combines multiple values into a single value.

For example, computing the mean of three numbers (for example `[0, 1, 2]`) returns a single number (1).

We will use aggregations extensively as we analyze and manipulate our data.

In [ ]:
# Built-in aggregations
unemp.mean()

In [ ]:
unemp.max(axis=1)

In [ ]:
# Write a custom aggregation
def high_or_low(s):
    """Classify unemployment as high (>6.5) or low"""
    return "High" if s.mean() > 6.5 else "Low"

unemp.agg(high_or_low)

### Transforms

Many analytical operations do not necessarily involve an aggregation.

The output of a function applied to a Series might need to be a new
Series.

Some examples:

- Compute the percentage change in unemployment from month to month.  
- Calculate the cumulative sum of elements in each column.  

In [ ]:
# Built in transforms
unemp.pct_change().head()

In [ ]:
unemp.diff().head()

In [ ]:
# Custom transform
def standardize(x):
    return (x - x.mean()) / x.std()

std_unemp = unemp.apply(standardize)
std_unemp.head()

There is a "special case" of transform where what you're actually interested in doing is transforming each number independent of the other numbers -- Something like taking the absolute value takes multiple values into multiple values but doesn't have inter-data dependencies

In [ ]:
# Built-in scalar transform
unemp.abs()

In [ ]:
# Custom scalar transform
def unemployment_classifier(ur):
    """
    Classifies the unemployment rate as high, medium, or low
    based on the value
    
    Parameters
    ----------
    ur : scalar(float)
        The unemployment rate
    
    Returns
    -------
    out : str
        The classification "high", "medium", or "low"
    """
    if ur > 6.5:
        return "high"
    elif ur > 4.5:
        return "medium"
    else:
        return "low"

unemp_bins = unemp.map(unemployment_classifier)
unemp_bins.head()

**Boolean selection + query**

Above we saw how we can select specific rows/columns or select by certain dates, but we may also need to select data based on conditions met by the data itself.

Some examples are:

- Restrict analysis to all individuals older than 18.  
- Look at data that corresponds to particular time periods.  
- Analyze only data that corresponds to a recession.  
- Obtain data for a specific product or customer ID.  

We will be able to do this by using a Series or list of boolean values to index into a Series or DataFrame.

Let’s look at some examples.

In [ ]:
unemp["NorthEast"] < 4.5

In [ ]:
unemp.loc[unemp["NorthEast"] < 4.5, :]

In [ ]:
unemp.query("NorthEast < 4.5")

In [ ]:
unemp.loc[(unemp["NorthEast"] > 6.5) | (unemp["West"] > 6.5), :]

In [ ]:
unemp.loc[(unemp["NorthEast"] > 6.5) & (unemp["West"] > 6.5), :]

In [ ]:
unemp.loc[(unemp < 6.5).all(axis=1), :]

### The Index

We told you that the index was the “row labels” for the data

This is true, but an index in pandas does much more than label the rows

The [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html) says

> Data alignment is intrinsic. The link between labels and data will not be broken unless done so explicitly by you.


In practice, the index and column names are used to make sure the data is properly aligned when operating on multiple DataFrames

In [ ]:
# Load WDI data
url = "https://datascience.quantecon.org/assets/data/wdi_data.csv"
wdi_raw = pd.read_csv(url)

To motivate the index, let's grab a subset of this data and see what happens when we "add" some columns of two different DataFrames

In [ ]:
df_small = wdi_raw.head(5)
df_small

In [ ]:
df_tiny = wdi_raw.iloc[[0, 3, 2, 4], :]
df_tiny

In [ ]:
df_small.loc[:, ["Exports", "Imports"]] + df_tiny.loc[:, ["Exports", "Imports"]]

For all (row, column) combinations that appear in both DataFrames (e.g. rows `[1, 3]` and columns `[Imports, Exports]`), the value of `im_ex_tiny` is equal to `df_tiny.loc[row, col] + im_ex.loc[row, col]`

This happened even though the rows and columns were not in the same order

We refer to this as pandas *aligning* the data for us

To see how awesome this is, think about how to do something similar in
Excel:

- `df_tiny` and `im_ex` would be in different sheets
- The index and column names would be the first column and row in each sheet
- We would have a third sheet to hold the sum
- For each label in the first row and column of *either* the `df_tiny` sheet or the `im_ex` sheet we would have to do a `IFELSE` to check if the label exists in the other sheet and then a `VLOOKUP` to extract the value

**Setting and resetting an index**

You can choose which index to use -- The choice of index depends on the question at hand

In [ ]:
wdi_country = wdi_raw.set_index("country")

In [ ]:
wdi_country.loc["Canada", :]

In [ ]:
wdi_year = wdi_raw.set_index("year")

In [ ]:
wdi_year.loc[2015, :]

You can also choose multiple columns to be your index:

In [ ]:
wdi = wdi_raw.set_index(["country", "year"])

In [ ]:
wdi.head()

A few more rules for how indexing into a hierarchical index works...

In [ ]:
wdi.loc[("Canada", 2015), :]

In [ ]:
wdi.loc[(["Canada", "United States"], [2015, 2017]), :]

In [ ]:
wdi.loc[pd.IndexSlice[:, [2005, 2007, 2009]], :]

### Split-Apply-Combine (groupby)

One powerful paradigm for analyzing data is the “Split-Apply-Combine” strategy

This strategy has three steps:

1. `Split`: split the data into groups based on values in one or more columns.  
2. `Apply`: apply a function or routine to each group separately.  
3. `Combine`: combine the output of the apply step into a DataFrame, using the group identifiers as the index

We will cover the core concepts here

We **strongly** encourage you to also study the [official documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

We are going to start with a "toy dataset"

In [ ]:
C = np.arange(1, 7, dtype=float)
C[[3, 5]] = np.nan
df = pd.DataFrame({
    "A" : [1, 1, 1, 2, 2, 2],
    "B" : [1, 1, 2, 2, 1, 1],
    "C": C,
})
df

In [ ]:
gbA = df.groupby("A")

In [ ]:
type(gbA)

We can extract groups from the GroupBy object

In [ ]:
gbA.get_group(1)

We can also apply aggregations to the GroupBy object which will apply the aggregation to each possible group

In [ ]:
gbA.sum()

Just like we could with the standard DataFrame object, we can apply custom aggregations/transformations to each group through the GroupBy object

In [ ]:
def num_missing(df):
    "Return the number of missing items in each column of df"
    return df.isnull().sum()

df.groupby("A").agg(num_missing)

In [ ]:
def smallest_by_b(df):
    return df.nsmallest(2, "B")

df.groupby("A").apply(smallest_by_b, include_groups=False)

In [ ]:
# write function here
def deviation_from_mean(x):
    """
    Compute the deviation of each value of x from its mean
    
    Parameters
    ----------
    x: pd.Series, pd.DataFrame
        The Series or DataFrame for which to do the computation
    
    Returns
    -------
    x_hat: type(x)
        The transformed version of x
    """
    return x - x.mean()
    


# apply function here
deviations = df.groupby("A").apply(deviation_from_mean, include_groups=False)
deviations

There is a special helper, similar to `pd.IndexSlice` that allows us to do "special things" to the groups. This is especially helpful for resampling dates!

In [ ]:
df2 = df.copy()
df2["Date"] = pd.date_range(
    start=dt.datetime.today().strftime("%m/%d/%Y"),
    freq="BQE",
    periods=df.shape[0]
)
df2 = df2.set_index("A")
df2


In [ ]:
df2.groupby(pd.Grouper(key="Date", freq="YE")).count()

### Reshaping data


While pushed more generally in the `R` language, the concept of “[tidy data](https://en.wikipedia.org/wiki/Tidy_data)” is helpful in understanding the objectives for reshaping data, which in turn makes advanced features like GroupBy more seamless.

Hadley Wickham gives a terminology slightly better-adapted for the experimental sciences, but nevertheless useful for the social sciences.

> A dataset is a collection of values, usually either numbers (if
quantitative) or strings (if qualitative). Values are organized in two
ways. Every value belongs to a variable and an observation. A variable
contains all values that measure the same underlying attribute (like
height, temperature, duration) across units. An observation contains all
values measured on the same unit (like a person, or a day, or a race)
across attributes. – [Tidy Data (Journal of Statistical Software 2013)](https://www.jstatsoft.org/index.php/jss/article/view/v059i10/v59i10.pdf)

With this framing,

> A dataset is messy or tidy depending on how rows, columns and tables are matched with observations, variables, and types. In tidy data:

1.  Each variable forms a column.
2.  Each observation forms a row.
3.  Each type of observational unit forms a table.

The “column” and “row” terms map directly to pandas columns and rows, while the “table” maps to a pandas DataFrame.

The question that should come to mind anytime you're introduced to a dataset is, "What uniquely identifies an “observation” in your data?"

Is it a country? A year? A combination of country and year?

These will become the indices of your DataFrame.

The concept of an "observation" may  not be unique to a dataset. For example, consider a time-series of county level GDP data.

* The most "pure" form of tidy data would probably classify the year/country as the identifier and have a single variable of GDP
* You could also consider the year to be the unique identifier and have each country's GDP be the variable
* Or, a variable might even be the GDP in a given year with the countries being the unique identifiers

What you consider to be an observation will depend on the question you're asking.

**Tall vs Wide**

Data can either be "tall" or "wide"

If we take our WDI data from above, it was a relatively "wide" dataset

In [ ]:
wdi_raw.head()

This is what a "tall" version of that dataset would look like

In [ ]:
wdi_raw.melt(id_vars=["country", "year"])

**Reshaping operations**

The core reshaping operations are:

* `.set_index`
* `.reset_index`
* `.stack`
* `.unstack`

Additionally, these get combined into some convenience reshaping functions called

* `melt`
* `pivot`
* `pivot_table`


**`stack` and `unstack`**

`stack` brings a "column index" into the "row index" and `unstack` brings a "row index" into a column index.

Let's see some examples

In [ ]:
df = wdi_raw.set_index(["country", "year"])

In [ ]:
df.unstack(level="year")

In [ ]:
df.unstack(level=0)

In [ ]:
df.stack(level=0)

Why might we want to have our data be stacked or unstacked in a particular way? Because it might make it easier to answer certain questions. For example, imagine that we wanted the average of each variable for each country over our time-series, then we could do

In [ ]:
df.unstack(level="country").mean(axis=0)

**`melt` and `pivot`**

It turns out that `melt` and `pivot` could be created using just the operations that you've already been taught (a great way to prove to yourself that you understand these operations is to reconstruct them from scratch!), but they are common enough operations that the convenience is helpful

We already saw that `melt` takes a wide DataFrame and turns it into a tall DataFrame

In [ ]:
wdi_tall = wdi_raw.melt(
    id_vars=["country", "year"], value_vars=["GovExpend", "GDP"], var_name="variable", value_name="value"
)

wdi_tall.head()

`pivot`/`pivot_table` can be thought of as basically the reverse of `melt`. It takes a tall DataFrame and makes it wide.



In [ ]:
wdi_tall.pivot(columns="variable", index=["country", "year"], values="value")

In [ ]:
wdi_tall.pivot_table(columns="variable", index=["country", "year"], values="value")

They appear to do the same thing and they mostly do. `pivot_table` method is actually a slight generalization of `pivot`.

It overcomes one limitation of `pivot`: It allows you to deal with duplicate entries by having you choose how to combine them.

* I use `pivot` when I don't want multiple values to be put into the same cell.
* I use `pivot_table` when I expect multiple values to go into a cell and specify an aggregation

In [ ]:
wdi_tall.pivot(index="country", columns="variable", values="value")

In [ ]:
wdi_tall.pivot_table(index="country", columns="variable", values="value", aggfunc="min")

Here are some gifs for each of the operations that might help deepen your understanding

**`stack`**

<img src="https://datascience.quantecon.org/_images/stack.gif" alt="stack.gif" style="">

**`unstack`**

<img src="https://datascience.quantecon.org/_images/unstack_level0.gif" alt="unstack\_level0.gif" style="">

**`melt`**

<img src="https://datascience.quantecon.org/_images/melt.gif" alt="melt.gif" style="">

### Merge datasets

We often need to combine data from multiple sources. pandas provides three main tools:

1. `pd.concat`: Stack DataFrames
2. `pd.merge`: Combine by matching keys (like SQL joins)
3. `df.join`: Convenient wrapper for merge

Let's work with country-level data.

In [ ]:
# Create sample datasets
sq_miles = pd.Series({
    "United States": 3.8,
    "Canada": 3.8,
    "Germany": 0.137,
    "United Kingdom": 0.0936,
    "Russia": 6.6,
}, name="sq_miles").to_frame()
sq_miles.index.name = "country"

# Population data
pop_url = "https://datascience.quantecon.org/assets/data/wdi_population.csv"
pop = pd.read_csv(pop_url).set_index(["country", "year"])

print("Square miles:")
display(sq_miles)
print("\nPopulation (first few):")
display(pop.head())

In [ ]:
wdi_2017 = wdi_raw.query("year == 2017").set_index("country")

In [ ]:
sq_miles

In [ ]:
pop

**pd.concat - stacking DataFrames:**

In [ ]:
# Stack side by side (axis=1)
pd.concat([wdi_2017, sq_miles], axis=1)

**pd.merge - combining with keys:**

In [ ]:
# Merge on country
pd.merge(wdi_2017, sq_miles, on="country")

**Different join types:**

- `how="left"`: Keep all rows from left DataFrame (default)
- `how="right"`: Keep all rows from right DataFrame
- `how="inner"`: Keep only matching rows
- `how="outer"`: Keep all rows from both

In [ ]:
# Outer join - keeps Russia even though not in wdi_2017
pd.merge(wdi_2017, sq_miles, on="country", how="outer")

**Merging on multiple keys:**

In [ ]:
# Get 2016-2017 data
wdi_recent = wdi.loc[pd.IndexSlice[:, [2016, 2017]], :].reset_index()

# Merge with population on both country and year
wdi_with_pop = pd.merge(
    wdi_recent,
    pop.reset_index(),
    on=["country", "year"]
)

# Calculate GDP per capita
wdi_with_pop["GDP_per_capita"] = wdi_with_pop["GDP"] / wdi_with_pop["Population"]
wdi_with_pop[["country", "year", "GDP", "Population", "GDP_per_capita"]]

## `matplotlib`

matplotlib is Python's main plotting library. pandas integrates with it seamlessly.

We introduce here briefly but John has largely convinced me that there's no reason for me to write plotting code in most cases anymore... Claude typically (1) has better design taste than me (2) when it doesn't, I can just tell it why the graph is ugly and it will fix it, and (3) it remembers more about `matplotlib` than I've ever known.

### Basic plotting

In [ ]:
# Simple line plot
x = np.linspace(0, 2*np.pi, 100)

fig, ax = plt.subplots()

ax.plot(x, np.sin(x))
ax.set_xlabel('x')
ax.set_ylabel('sin(x)')
ax.set_title('Sine Wave')

### Plotting with `pandas`

pandas DataFrames have a `.plot()` method that creates matplotlib figures:

In [ ]:
# Line plot of unemployment
unemp.plot(figsize=(10, 6), title="Unemployment by Region")

In [ ]:
# Bar plot
unemp.loc[2009].plot(kind="bar", title="2009 Unemployment by Region")

### Advanced(ish) plotting

**Subplots** let you create multiple plots in one figure:

In [ ]:
# Create 2x2 subplots for each region
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

unemp["NorthEast"].plot(ax=axes[0, 0], title="NorthEast")
unemp["MidWest"].plot(ax=axes[0, 1], title="MidWest")
unemp["South"].plot(ax=axes[1, 0], title="South")
unemp["West"].plot(ax=axes[1, 1], title="West")

plt.tight_layout()

**Scatter plots** for relationships:

In [ ]:
# Scatter: Consumption vs GDP
wdi_2017.plot.scatter(
    x="GDP",
    y="Consumption",
    title="Consumption vs GDP (2017)"
)